In [9]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# hyperparameters
n_samples = 1000
seq_len = 20
dynamic_feat_dim = 2
static_feat_dim = 1
num_classes = 3

# Simulated data
X_dynamic = torch.randn(n_samples, seq_len, dynamic_feat_dim)
X_static = torch.randn(n_samples, static_feat_dim)
y = torch.randint(0, num_classes, (n_samples,))  # classification labels

# Train/test split
train_size = int(0.8 * n_samples)
X_dyn_train, X_dyn_test = X_dynamic[:train_size], X_dynamic[train_size:]
X_sta_train, X_sta_test = X_static[:train_size], X_static[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [15]:
X_dynamic.shape, X_static.shape

(torch.Size([1000, 20, 2]), torch.Size([1000, 1]))

In [10]:
counts = Counter(y_train.tolist())
print(counts)

# Step 2: Create weights (inverse frequency)
total = sum(counts.values())
weights = [total / counts[i] for i in range(num_classes)]
class_weights = torch.tensor(weights, dtype=torch.float)

Counter({0: 276, 1: 266, 2: 258})


In [11]:
class_weights

tensor([2.8986, 3.0075, 3.1008])

In [ ]:
# Model for classification
class LSTMWithStaticClassifier(nn.Module):
    def __init__(self, input_size, static_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size + static_size, num_classes)

    def forward(self, x_dyn, x_static):
        lstm_out, _ = self.lstm(x_dyn)  # [batch, seq_len, hidden]
        last_hidden = lstm_out[:, -1, :]  # [batch, hidden]
        combined = torch.cat([last_hidden, x_static], dim=1)
        logits = self.fc(combined)  # [batch, num_classes]
        return logits

model = LSTMWithStaticClassifier(
    input_size=dynamic_feat_dim,
    static_size=static_feat_dim,
    hidden_size=64,
    num_layers=1,
    num_classes=num_classes
)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(200):
    model.train()
    logits = model(X_dyn_train, X_sta_train)
    loss = criterion(logits, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        pred_labels = logits.argmax(dim=1)
        acc = (pred_labels == y_train).float().mean()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Train Acc: {acc:.4f}")

Epoch 10, Loss: 1.0967, Train Acc: 0.3663
Epoch 20, Loss: 1.0940, Train Acc: 0.3775
Epoch 30, Loss: 1.0909, Train Acc: 0.3750
Epoch 40, Loss: 1.0872, Train Acc: 0.3887
Epoch 50, Loss: 1.0831, Train Acc: 0.4087
Epoch 60, Loss: 1.0772, Train Acc: 0.4263
Epoch 70, Loss: 1.0708, Train Acc: 0.4162
Epoch 80, Loss: 1.0600, Train Acc: 0.4437
Epoch 90, Loss: 1.0451, Train Acc: 0.4625
Epoch 100, Loss: 1.0277, Train Acc: 0.4600
Epoch 110, Loss: 1.0081, Train Acc: 0.4888
Epoch 120, Loss: 0.9886, Train Acc: 0.5150
Epoch 130, Loss: 0.9586, Train Acc: 0.5412
Epoch 140, Loss: 0.9332, Train Acc: 0.5537
Epoch 150, Loss: 0.9247, Train Acc: 0.5562
Epoch 160, Loss: 0.8914, Train Acc: 0.5850
Epoch 170, Loss: 0.8540, Train Acc: 0.6037
Epoch 180, Loss: 0.8272, Train Acc: 0.6388
Epoch 190, Loss: 0.7979, Train Acc: 0.6612
Epoch 200, Loss: 0.7755, Train Acc: 0.6575


In [13]:
# Evaluation
model.eval()
with torch.no_grad():
    test_logits = model(X_dyn_test, X_sta_test)
    test_preds = test_logits.argmax(dim=1)
    test_acc = (test_preds == y_test).float().mean()
    print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.3700
